In [1]:
import os
os.chdir("/Users/yenchenchou/Documents/GitHub/ml-learning")

In [3]:
import random
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import time
import torch
import torch.nn as nn

from math import sqrt
from numpy import loadtxt
from itertools import product
from tqdm import tqdm
from sklearn import preprocessing
from xgboost import plot_tree
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

kernel_with_output = False

from src import EnvInit

In [4]:
env_init = EnvInit()
device = env_init.available_device()
seed = env_init.fix_seed(13317)
device, seed

(device(type='mps'), 13317)

In [13]:
train = pl.read_csv("data/PredictFutureSales/sales_train.csv")
items = pl.read_csv("data/PredictFutureSales/items.csv")
shops = pl.read_csv("data/PredictFutureSales/shops.csv")
categories = pl.read_csv("data/PredictFutureSales/item_categories.csv")
test = pl.read_csv("data/PredictFutureSales/test.csv")
# sample_submission = pd.read_csv("data/PredictFutureSales/sample_submission.csv")

In [14]:
print(train.shape)
train.head(2)

(2935849, 6)


date,date_block_num,shop_id,item_id,item_price,item_cnt_day
str,i64,i64,i64,f64,f64
"""02.01.2013""",0,59,22154,999.0,1.0
"""03.01.2013""",0,25,2552,899.0,1.0


In [15]:
items.head(2)

item_name,item_id,item_category_id
str,i64,i64
"""! ВО ВЛАСТИ НА…",0,40
"""!ABBYY FineRea…",1,76


In [16]:
shops.head(2)

shop_name,shop_id
str,i64
"""!Якутск Орджон…",0
"""!Якутск ТЦ ""Це…",1


In [17]:
categories.head(2)

item_category_name,item_category_id
str,i64
"""PC - Гарнитуры…",0
"""Аксессуары - P…",1


In [18]:
test.head(2)

ID,shop_id,item_id
i64,i64,i64
0,5,5037
1,5,5320


In [ ]:
class Preprocessor:
    def get_monthly_data(self):
        pass

In [12]:
%%timeit
grid = []
for block_num in train['date_block_num'].unique():
    cur_shops = train[train['date_block_num']==block_num]['shop_id'].unique()
    cur_items = train[train['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
index_cols = ['shop_id', 'item_id', 'date_block_num']
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# # Aggregations
# sales_train['item_cnt_day'] = sales_train['item_cnt_day'].clip(0,20)
# groups = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'])
# trainset = groups.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()
# trainset = trainset.rename(columns = {'item_cnt_day' : 'item_cnt_month'})
# trainset['item_cnt_month'] = trainset['item_cnt_month'].clip(0,20)

# trainset = pd.merge(grid,trainset,how='left',on=index_cols)
# trainset.item_cnt_month = trainset.item_cnt_month.fillna(0)

# # Get category id
# trainset = pd.merge(trainset, items[['item_id', 'item_category_id']], on = 'item_id')
# trainset.to_csv('trainset_with_grid.csv')

# trainset.head()

2.33 s ± 9.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
